# A simple demo for the OpenHowNet Python Package

To begin with, make sure you have installed **Python 3.X**. 

Also, the [**anytree**](https://pypi.org/project/anytree/) is required to be installed, which is the only dependency for OpenHowNet.

Next, you should follow the [instruction](https://github.com/thunlp/OpenHowNet#installation) to install **OpenHowNet** API. 

After that, you can import the module:

In [ ]:
# Import the OpenHowNet module
import OpenHowNet

Then we can create a **HowNetDict** object:

In [ ]:
# Initialize HowNetDict, you can initialize the similarity calculation module by setting the init_sim to True.
hownet_dict = OpenHowNet.HowNetDict(init_sim=False)

Now the preparation work is all done. Let's explore some important features of HowNetDict.

## Basic Usage of OpenHowNet

### Get word annotations in HowNet

By default, the api will search the target word in both English and Chinese annotations in HowNet, which will cause significant search overhead. Note that if the target word does not exist in HowNet annotation, this api will simply return an empty list.

In [ ]:
# Get the senses list annotated with "苹果".
result_list = hownet_dict.get_sense("苹果")
print("The number of retrievals: ", len(result_list))
print("An example of retrievals: ", result_list)

In OpenHowNet package, the detailed information of senses and sememes in HowNet are wrapped into classes.

In [ ]:
# Get the detailed information of the sense.
sense_example = result_list[0]
print("Sense example:", sense_example)
print("Sense id: ",sense_example.No)
print("English word in the sense: ", sense_example.en_word)
print("Chinese word in the sense: ", sense_example.zh_word)
print("HowNet annotation of the sense: ", sense_example.Def)
print("Sememe list of the sense: ", sense_example.get_sememe_list())

In [ ]:
# Get the detailed information of the sememe.
sememe_example = sense_example.get_sememe_list().pop()
print("Sememe example: ", sememe_example)
print("The English annotation of the sememe: ", sememe_example.en)
print("The Chinese annotation of the sememe: ", sememe_example.zh)
print("The frequency of occurrence of the sememe in HowNet: ", sememe_example.freq)

You can visualize the retrieved HowNet structured annotations ("sememe tree") of sense as follow :

In [ ]:
sense_example.visualize_sememe_tree()

Besides, you can get the Sememe instance list by the English annotation or Chinese annotation. Similarily, you can set the language of the input or set the `strict` to `False` to fuzzy match the sememe.

In [ ]:
sememe1 = hownet_dict.get_sememe('FormValue', language='en')
sememe2 = hownet_dict.get_sememe('圆', language='zh')
print("Retrieved sememes: ",sememe1, sememe2)

sememe3 = hownet_dict.get_sememe('值', strict=False)
print("Fuzzy match the sememes (retrieved {} results): ".format(len(sememe3)), sememe3[:5])

sememe_all = hownet_dict.get_all_sememes()
print("There are {} sememes in HowNet in total.".format(len(sememe_all)))

To boost the efficiency of the search process, you can specify the language of the target word as the following.

In [ ]:
print("The number of mixed search results:",len(hownet_dict.get_sense("X")))
print("The number of Chinese results:",len(hownet_dict.get_sense("X",language="zh")))
print("The number of English results:",len(hownet_dict.get_sense("X",language="en")))

You can limit the POS of the target word by setting the `pos`.  Besides, you can set the `strict` to false to make a fuzzy match.

In [ ]:

res = hownet_dict.get_sense("苹果", strict=False)
print("Fuzzy match: (The number of retrievals: {})".format(len(res)))
print("Retrivals: {}\n".format(res))
res = hownet_dict.get_sense("苹果",pos='adj', strict=False)
print("Fuzzy match and limit the POS to adj: (The number of retrievals: {})".format(len(res)))
print("Retrivals: {}".format(res))


You can get all senses by using the follow API.

In [ ]:
all_senses = hownet_dict.get_all_senses()
print("The number of all senses: {}".format(len(all_senses)))

Besides, you can also get all the English or Chinese words in HowNet annotations.

In [ ]:
zh_word_list = hownet_dict.get_zh_words()
en_word_list = hownet_dict.get_en_words()
print("Chinese words in HowNet: ",zh_word_list[:10])
print("English words in HowNet: ",en_word_list[:10])

### Get sememe trees of a certain word in HowNet¶

You can get the sememes by certain word in a variety of forms of presentation. Detailed explanation of params will be displayed in our documentation.
First, you can retrieve all the senses annotated with the certain word and their sememes.

In [ ]:
# Get the respective sememe list of the senses annotated with the word.
# The word can be English or Chinese or *
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=False, expanded_layer=-1, K=None)

The `display` can be set to "tree"/"dict"/"list"/"visual", and the function will return in different forms.
1. When set to "list", the sememes will be returned in the form of list as shown above.
2. When set to "dict", the function will return the sememe tree in the form of dict as shown below.

In [ ]:
hownet_dict.get_sememes_by_word(word = '苹果', display='dict', merge=False, expanded_layer=-1, K=None)[0]

3. When set to "tree", the function will return the senses and the root node of their respective sememe tree. 

In [ ]:
t = hownet_dict.get_sememes_by_word(word = '苹果', display='tree', merge=False, expanded_layer=-1, K=None)[0]
print(t)
print("The type of the root node is:", type(t['sememes']))

4. When set to "visual", the function will visualize the Top-K sememe trees. At this point, `K` can be set to control the num of the visualized sememe trees. 

In [ ]:
hownet_dict.get_sememes_by_word(word = '苹果', display='visual', merge=False, expanded_layer=-1, K=2)

5. `merge` and `expanded_layer` only work when `display=="list"`. When `merge==True`, the sememe lists of all the senses retrieved by the word will be merged into one. `expanded_layer` is set to control the expanded layer num of the sememe tree and by default it will be set to -1(expanded all layers).

In [ ]:
# Expand all layers and merge all the sememe list into one
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=True, expanded_layer=-1, K=None)

In [ ]:
# Expand the top2 layers and merge all the sememe list into one. Note that the first layer is the sense node. 
hownet_dict.get_sememes_by_word(word = '苹果', display='list', merge=True, expanded_layer=2, K=None)

### Get sememes via relations between sememes

There are various relations between sememes as follows. The package provides api to retrieve related sememes.
You can retrieve the relation between two sememes by the annotation of the sememe.

In [ ]:
all_sememe_relations = hownet_dict.get_all_sememe_relations()
print(all_sememe_relations)
# Get the relation between sememes. Please pay attention to the order of the sememes.
relations = hownet_dict.get_sememe_relation('FormValue','圆', return_triples=False)
print(relations)
# You can get the triples in the form of (head_sememe, relation, tail_relation)
triples = hownet_dict.get_sememe_relation('FormValue','圆', return_triples=True)
print(triples)

If you want sememes that have the exact relation with some sememe, you can do as below. Note that you can also get triples.

In [ ]:
triples = hownet_dict.get_related_sememes('FormValue', relation='hyponym',return_triples=True)
print(triples)

Besides, you can get related sememes directly by the sememe instance.

In [ ]:
print("Take {} as example.".format(sememe1[0]))
print("The sememes that have the relaiton of hyponym with the sememe are:")
print(sememe1[0].get_related_sememes(relation='hyponym'))

Moreover, you can get all the sememes that have relation with the exact sememe (ignore the order).

In [ ]:
print("The sememes that have relaiton with the sememe {} are:".format(sememe1[0]))
print(sememe1[0].get_related_sememes())

## Advanced Feature #1: Word Similarity Calculation via Sememes

The following parts are mainly implemented by Jun Yan and integrated by Chenghao Yang. Our implementation is based on the paper:
> Jiangming Liu, Jinan Xu, Yujie Zhang. An Approach of Hybrid Hierarchical Structure for Word Similarity Computing by HowNet. In Proceedings of IJCNLP

### Extra initializaiton
Because there are some files required to be loaded for similarity calculation. The initialization overhead will be larger than before. To begin with, you can initialize the hownet_dict object as the following code :

In [ ]:
hownet_dict_anvanced = OpenHowNet.HowNetDict(init_sim=True)

You can also postpone the initialization work of similarity calculation until use. The following code serves as an example.

In [ ]:
hownet_dict.initialize_similarity_calculation()

### Get senses that have the same sememe list
You can retrieve the senses that have the same sememe list with the exact sense. Note that the structured information is ignored.

In [ ]:
print("Take sense {} as an example. Its sememes contains: ".format(sense_example))
print(sense_example.get_sememe_list())
print("Senses that have the same sememe list contains: ")
print(hownet_dict_anvanced.get_sense_synonyms(sense_example)[:10])

### Get top-K nearest words of the given word
Given an exact word, the function will return the Top-K nearest words in HowNet.
First of all, the HowNetDict will match the senses in HowNet by the word and give the nearest words separately.
Note that you must set the language of the words, and the calculation may takes a long time.

In [ ]:
hownet_dict_anvanced.get_nearest_words('苹果', language='zh',K=5)

You can get the similarity score as below:

In [ ]:
hownet_dict_anvanced.get_nearest_words('苹果', language='zh',K=5,score=True)

By setting the `merge` to True, you can merge the words list of senses into one and get the Top-K words.

In [ ]:
hownet_dict_anvanced.get_nearest_words('苹果', language='zh',K=5, merge=True)

Detailed explanation of params will be displayed in our documentation.

### Calculate the similarity between two given words¶
If any of the given words does not exist in HowNet annotations, this function will return -1.

In [ ]:
print('The similarity of 苹果 and 梨 is {}.'.format(hownet_dict_anvanced.calculate_word_similarity('苹果','梨')))

## Advanced Feature #2: BabelNet Synset Search

### Extra initializaiton
Because there are more files required to be loaded for BabelNet dict. The initialization overhead will be larger than before. You can initialize the hownet_dict object as the following code :

In [ ]:
hownet_dict_anvanced = OpenHowNet.HowNetDict(init_babel=True)

Or you can use the following API to initialize the BabelNet dict.

In [ ]:
hownet_dict.initialize_babelnet_dict()

You can retrieve a synset instance and get the abundant information in it using the follow APIs.

In [ ]:
syn_list = hownet_dict_anvanced.get_synset('黄色')
print("{} results are retrieved and take the first one as an example".format(len(syn_list)))
syn_example = syn_list[0]
print("Synset: {}".format(syn_example))
print("English synonyms: {}".format(syn_example.en_synonyms))
print("Chinese synonyms: {}".format(syn_example.zh_synonyms))
print("English glosses: {}".format(syn_example.en_glosses))
print("Chinese glosses: {}".format(syn_example.zh_glosses))

You can also get all the synsets and relations between synsets:

In [ ]:
all_synsets = hownet_dict_anvanced.get_all_babel_synsets()
all_synset_relation = hownet_dict_anvanced.get_all_synset_relations()
print("There are {} synsets and {} relations".format(len(all_synsets),len(all_synset_relation)))

Also, you can search for the synsets that have the exact relation with the synset.

In [ ]:
related_synsets = syn_example.get_related_synsets()
print("There are {} synsets that have relation with the {}, they are: ".format(len(related_synsets), syn_example))
print(related_synsets[:10])

The package also provides search for the sememe list by the BabelNet sememe annotations.
The API is similar with the HowNet APIs.

In [ ]:
print(hownet_dict_anvanced.get_sememes_by_word_in_BabelNet('黄色'))
print(hownet_dict_anvanced.get_sememes_by_word_in_BabelNet('黄色',merge=True))